In [320]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain
from more_itertools import unique_everseen
import sets
import time

In [321]:
songs = pd.read_csv('FinalData_1.csv')
songs = songs[songs['Year']>=1960]
songs.head()

,Artist,Track,Year,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Marty Robbins,El Paso,1960,0.654,0.452,2,-9.709,1,0.0300,0.835,0.000029,0.160,0.691,106.662
1,Frankie Avalon,Why,1960,0.510,0.349,5,-8.677,1,0.0264,0.762,0.000000,0.130,0.586,94.267
2,Miss Toni Fisher,The Big Hurt,1960,0.624,0.557,0,-8.309,1,0.0299,0.783,0.000000,0.173,0.676,123.919
3,Johnny Preston,Running Bear,1960,0.772,0.297,5,-14.679,0,0.0530,0.854,0.000008,0.125,0.822,119.987
4,Guy Mitchell,Heartaches By The Number,1960,0.584,0.589,5,-11.203,1,0.0718,0.463,0.000015,0.069,0.842,172.484


In [322]:
songs.shape

(15519, 14)

In [323]:
songs.dtypes

Artist               object
Track                object
Year                  int64
Danceability        float64
Energy              float64
Key                   int64
Loudness            float64
Mode                  int64
Speechiness         float64
Acousticness        float64
Instrumentalness    float64
Liveness            float64
Valence             float64
Tempo               float64
dtype: object

In [324]:
songs.describe()

,Year,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
count,15519.000000,15519.000000,15519.000000,15519.000000,15519.000000,15519.000000,15519.000000,15519.000000,15519.000000,15519.000000,15519.000000,15519.000000
mean,1985.016238,0.594332,0.619774,5.235324,-8.981410,0.743282,0.063068,0.293600,0.036902,0.192688,0.621229,120.481247
std,17.467120,0.149736,0.200821,3.546958,3.608166,0.436836,0.069844,0.278679,0.144187,0.163888,0.236036,27.478903
min,1960.000000,0.000000,0.020100,0.000000,-28.030000,0.000000,0.000000,0.000002,0.000000,0.013000,0.000000,0.000000
25%,1969.000000,0.497000,0.473000,2.000000,-11.369000,0.000000,0.031600,0.048100,0.000000,0.088400,0.442000,100.065500
50%,1982.000000,0.603000,0.635000,5.000000,-8.630000,1.000000,0.039200,0.199000,0.000009,0.130000,0.649000,118.999000
75%,2000.000000,0.699000,0.782000,8.000000,-6.182000,1.000000,0.058600,0.500000,0.000803,0.250000,0.820000,135.721000
max,2019.000000,0.988000,0.997000,11.000000,2.291000,1.000000,0.950000,0.995000,0.982000,0.999000,0.990000,241.009000


In [325]:
songs = songs.drop(['Artist','Track'], axis = 1)

In [326]:
songs['Danceability'] = pd.cut(songs['Danceability'], bins=[-1, 0.33, 0.66, float('Inf')], labels=['Low Danceability', 'Medium Danceability', 'High Danceability'])

In [327]:
songs['Energy'] = pd.cut(songs['Energy'], bins=[0, 0.5, float('Inf')], labels=['Low Energy', 'High Energy'])

In [328]:
songs['Instrumentalness'] = pd.cut(songs['Instrumentalness'], bins=[-1, 0.5, 1.0], labels=['Low Instrumentalness', 'High Instrumentalness'])

In [329]:
songs['Acousticness'] = pd.cut(songs['Acousticness'], bins=[-1, 0.33, 0.66, float('Inf')], labels=['Low Acousticness', 'Medium Acousticness' ,'High Acousticness'])

In [330]:
songs['Loudness'] = pd.cut(songs['Loudness'], bins=[-60, -8, float('Inf')], labels=['Loud', 'Soft'])

In [331]:
songs['Speechiness'] = pd.cut(songs['Speechiness'], bins=[-1, 0.031, 0.038, float('Inf')], labels=['Less Vocal', 'Music & Speech', 'More Vocal'])

In [332]:
songs['Liveness'] = pd.cut(songs['Liveness'], bins=[-1, 0.8, float('Inf')], labels=['No Audience', 'Live Audience'])

In [333]:
songs['Valence'] = pd.cut(songs['Valence'], bins=[-1, 0.5 , float('Inf')], labels=['Sad', 'Cheerful'])

In [334]:
songs['Tempo'] = pd.cut(songs['Tempo'], bins=[-1, 100, 135, float('Inf')], labels=['Low Tempo', 'Medium Tempo', 'High Tempo'])

In [335]:
for idx, row in songs.iterrows():
    if  songs.loc[idx,'Mode'] == 0:
        songs.loc[idx,'Mode'] = 'MinorModality'
    elif songs.loc[idx,'Mode'] == 1:
        songs.loc[idx,'Mode'] = 'MajorModality'

In [336]:
for idx, row in songs.iterrows():
    if  songs.loc[idx,'Key'] == 0:
        songs.loc[idx,'Key'] = 'C'
    elif songs.loc[idx,'Key'] == 1:
        songs.loc[idx,'Key'] = 'C#'
    elif songs.loc[idx,'Key'] == 2:
        songs.loc[idx,'Key'] = 'D'
    elif songs.loc[idx,'Key'] == 3:
        songs.loc[idx,'Key'] = 'D#'
    elif songs.loc[idx,'Key'] == 4:
        songs.loc[idx,'Key'] = 'E'
    elif songs.loc[idx,'Key'] == 5:
        songs.loc[idx,'Key'] = 'F'
    elif songs.loc[idx,'Key'] == 6:
        songs.loc[idx,'Key'] = 'F#'
    elif songs.loc[idx,'Key'] == 7:
        songs.loc[idx,'Key'] = 'G'
    elif songs.loc[idx,'Key'] == 8:
        songs.loc[idx,'Key'] = 'G#'
    elif songs.loc[idx,'Key'] == 9:
        songs.loc[idx,'Key'] = 'A'
    elif songs.loc[idx,'Key'] == 10:
        songs.loc[idx,'Key'] = 'A#'
    else:
        songs.loc[idx,'Key'] = 'B'

In [337]:
songs_6070 = songs[(songs['Year']>=1960) & (songs['Year'] <1970)]
songs_7080 = songs[(songs['Year']>=1970) & (songs['Year'] <1980)]
songs_8090 = songs[(songs['Year']>=1980) & (songs['Year'] <1990)]
songs_9000 = songs[(songs['Year']>=1990) & (songs['Year'] <2000)]
songs_0010 = songs[(songs['Year']>=2000) & (songs['Year'] <2010)]
songs_1020 = songs[(songs['Year']>=2010) & (songs['Year'] <2020)]

In [338]:
songs_6070 = songs_6070.drop(['Year'], axis = 1)
songs_7080 = songs_7080.drop(['Year'], axis = 1)
songs_8090 = songs_8090.drop(['Year'], axis = 1)
songs_9000 = songs_9000.drop(['Year'], axis = 1)
songs_0010 = songs_0010.drop(['Year'], axis = 1)
songs_1020 = songs_1020.drop(['Year'], axis = 1)

songs_6070.to_csv('songs_6070.csv',header = False, index = False)
songs_7080.to_csv('songs_7080.csv',header = False, index = False)
songs_8090.to_csv('songs_8090.csv',header = False, index = False)
songs_9000.to_csv('songs_9000.csv',header = False, index = False)
songs_0010.to_csv('songs_0010.csv',header = False, index = False)
songs_1020.to_csv('songs_1020.csv',header = False, index = False)


In [339]:
def combinations(lst,n):
    if n==0:
        return [[]]
    l=[]
    for i in range(0,len(lst)):
        m=lst[i]
        remLst=lst[i+1:]
        for p in combinations(remLst,n-1):
            l.append([m]+p)
    return l

In [340]:
def preprocessing(data):
    lis = []
    for row in data: 
        row1 = ','.join([w.strip() for w in row.split(',')])
        row1 = row1.split(",")
        lis.append(row1)
    return lis    


def getC1(data):
    C1 = {}
    for trans in data:
        for item in trans:
            if item not in C1:
                C1[item] = 1
            else:
                C1[item] += 1           
    return C1        
    
    
def getCk(k,L):
    if k ==2:
        C = list(combinations(L,k))
        C = [tuple(l) for l in C]
        for c in C:
            if has_infreq_subsets(k,L,c):
                C.remove(c)
            else:
                pass
        return C

    else:    
        flat = [item for subt in L for item in subt]
        uflat = list(unique_everseen(flat))
        C = list(combinations(uflat,k))
        C = [tuple(l) for l in C]
        for c in C:
            if has_infreq_subsets(k,L,c):
                C.remove(c)
            else:
                pass
        return C
 
      
def getL1(dic, min_sup):
    L1 = []
    for item in dic:
        if float(dic[item])/float(n) > min_sup:
            L1.append(item)
    return L1        


def getLk(C,min_sup,data):
    C1 = {}
    C = [tuple(l) for l in C]
    for c in C:
        C1[c] = 0
    for obs in data:
        for c in C:
            if set(c).issubset(obs):
                C1[c] += 1
    
    L = []
    for c in C:
        if float(C1[c])/n > min_sup:
            L.append(c)

    return L

    
def has_infreq_subsets(k,L,c):
    S = list(combinations(L, k-1))
    S = [tuple(l) for l in S]
    for i in S:
        if i in L:
            return False
        else:
            return True


def FreqIS(k,data, min_sup):
    C1 = getC1(data)
    print("\nC1:\n ", C1)
    L1 = getL1(C1,min_sup)
    print("\nL1:\n ", L1)
    listofL = []
    while L1!= []:
        C = getCk(k,L1)
        print("\nC"+str(k),":\n",C)
        L1 = getLk(C,min_sup,data)
        print("\nL"+str(k),":\n",L1)
        listofL.append(L1)
        k+=1
    return listofL[0:len(listofL)-1]

In [341]:
data1 = open('songs_6070.csv', 'r')
predata = preprocessing(data1)
n = len(predata)


print("Enter minimum support: ")
min_sup = float(input())

start = time.time()
freqsets = FreqIS(2,predata,min_sup)
end = time.time()

print("Therefore frequent patterns are itemsets present in last non-empty Lk and all its subsets")
print("Time taken: " + str(end-start))        

Enter minimum support: 
0.6

C1:
  {'MinorModality': 647, 'Medium Acousticness': 1300, 'Cheerful': 2932, 'Low Acousticness': 1060, 'E': 287, 'High Instrumentalness': 222, 'F#': 144, 'C#': 253, 'A': 367, 'B': 200, 'Medium Danceability': 2678, 'Live Audience': 65, 'C': 531, 'G': 514, 'High Danceability': 814, 'Less Vocal': 973, 'High Energy': 2035, 'Low Instrumentalness': 3672, 'Sad': 962, 'Loud': 2921, 'MajorModality': 3247, 'Low Tempo': 1033, 'Medium Tempo': 1877, 'D': 404, 'D#': 182, 'F': 449, 'High Acousticness': 1534, 'Music & Speech': 1162, 'G#': 235, 'Low Energy': 1859, 'Low Danceability': 402, 'No Audience': 3829, 'Soft': 973, 'High Tempo': 984, 'More Vocal': 1759, 'A#': 328}

L1:
  ['Cheerful', 'Medium Danceability', 'Low Instrumentalness', 'Loud', 'MajorModality', 'No Audience']

C2 :
 [('Cheerful', 'Low Instrumentalness'), ('Cheerful', 'MajorModality'), ('Medium Danceability', 'Low Instrumentalness'), ('Medium Danceability', 'MajorModality'), ('Low Instrumentalness', 'Loud'), 

In [342]:
data1 = open('songs_7080.csv', 'r')
predata = preprocessing(data1)
n = len(predata)


print("Enter minimum support: ")
min_sup = float(input())

start = time.time()
freqsets = FreqIS(2,predata,min_sup)
end = time.time()

print("Therefore frequent patterns are itemsets present in last non-empty Lk and all its subsets")
print("Time taken: " + str(end-start))        

Enter minimum support: 
0.6

C1:
  {'MinorModality': 761, 'Medium Acousticness': 874, 'Cheerful': 2359, 'E': 290, 'High Instrumentalness': 125, 'No Audience': 3040, 'C#': 189, 'A': 417, 'B': 178, 'Medium Danceability': 1923, 'Live Audience': 58, 'C': 438, 'A#': 211, 'F#': 119, 'High Danceability': 981, 'Less Vocal': 699, 'High Energy': 2045, 'Low Instrumentalness': 2973, 'Sad': 739, 'Medium Tempo': 1696, 'Low Acousticness': 1744, 'Low Tempo': 690, 'Loud': 2338, 'D': 378, 'D#': 85, 'F': 291, 'High Acousticness': 480, 'Music & Speech': 776, 'G#': 132, 'Low Energy': 1053, 'Low Danceability': 194, 'MajorModality': 2337, 'Soft': 760, 'High Tempo': 712, 'More Vocal': 1623, 'G': 370}

L1:
  ['Cheerful', 'No Audience', 'Medium Danceability', 'High Energy', 'Low Instrumentalness', 'Loud', 'MajorModality']

C2 :
 [('Cheerful', 'Medium Danceability'), ('Cheerful', 'Low Instrumentalness'), ('Cheerful', 'MajorModality'), ('No Audience', 'High Energy'), ('No Audience', 'Loud'), ('Medium Danceability

In [343]:
data1 = open('songs_8090.csv', 'r')
predata = preprocessing(data1)
n = len(predata)


print("Enter minimum support: ")
min_sup = float(input())

start = time.time()
freqsets = FreqIS(2,predata,min_sup)
end = time.time()

print("Therefore frequent patterns are itemsets present in last non-empty Lk and all its subsets")
print("Time taken: " + str(end-start))        

Enter minimum support: 
0.6

C1:
  {'MinorModality': 772, 'Medium Acousticness': 433, 'Cheerful': 1979, 'E': 221, 'High Instrumentalness': 51, 'No Audience': 2573, 'Soft': 874, 'A': 365, 'Medium Danceability': 1421, 'Live Audience': 44, 'C': 341, 'Sad': 638, 'F#': 128, 'High Danceability': 1127, 'Less Vocal': 636, 'High Energy': 2071, 'D#': 63, 'Low Instrumentalness': 2566, 'A#': 166, 'Medium Tempo': 1606, 'Low Acousticness': 2071, 'Low Tempo': 421, 'Loud': 1743, 'D': 342, 'C#': 170, 'F': 225, 'High Acousticness': 113, 'Music & Speech': 748, 'G#': 105, 'Low Energy': 546, 'Low Danceability': 69, 'MajorModality': 1845, 'High Tempo': 590, 'More Vocal': 1233, 'G': 285, 'B': 206}

L1:
  ['Cheerful', 'No Audience', 'High Energy', 'Low Instrumentalness', 'Medium Tempo', 'Low Acousticness', 'Loud', 'MajorModality']

C2 :
 [('Cheerful', 'High Energy'), ('Cheerful', 'Medium Tempo'), ('Cheerful', 'Loud'), ('No Audience', 'High Energy'), ('No Audience', 'Medium Tempo'), ('No Audience', 'Loud'), ('

In [344]:
data1 = open('songs_9000.csv', 'r')
predata = preprocessing(data1)
n = len(predata)


print("Enter minimum support: ")
min_sup = float(input())

start = time.time()
freqsets = FreqIS(2,predata,min_sup)
end = time.time()

print("Therefore frequent patterns are itemsets present in last non-empty Lk and all its subsets")
print("Time taken: " + str(end-start))        

Enter minimum support: 
0.6

C1:
  {'MinorModality': 649, 'Medium Acousticness': 297, 'Cheerful': 1287, 'No Audience': 1974, 'E': 152, 'Loud': 1120, 'High Instrumentalness': 47, 'C#': 215, 'Low Instrumentalness': 1946, 'A': 188, 'Medium Danceability': 1009, 'Live Audience': 19, 'C': 210, 'A#': 139, 'F#': 164, 'High Danceability': 938, 'Less Vocal': 540, 'High Energy': 1533, 'Soft': 873, 'Sad': 706, 'Medium Tempo': 927, 'Low Acousticness': 1630, 'Low Tempo': 615, 'Music & Speech': 410, 'D': 169, 'D#': 53, 'F': 165, 'High Acousticness': 66, 'G#': 130, 'Low Energy': 460, 'Low Danceability': 46, 'MajorModality': 1344, 'High Tempo': 451, 'More Vocal': 1043, 'G': 252, 'B': 156}

L1:
  ['Cheerful', 'No Audience', 'Low Instrumentalness', 'High Energy', 'Low Acousticness', 'MajorModality']

C2 :
 [('Cheerful', 'Low Instrumentalness'), ('Cheerful', 'Low Acousticness'), ('No Audience', 'Low Instrumentalness'), ('No Audience', 'Low Acousticness'), ('Low Instrumentalness', 'High Energy'), ('Low Ins

In [345]:
data1 = open('songs_0010.csv', 'r')
predata = preprocessing(data1)
n = len(predata)


print("Enter minimum support: ")
min_sup = float(input())

start = time.time()
freqsets = FreqIS(2,predata,min_sup)
end = time.time()

print("Therefore frequent patterns are itemsets present in last non-empty Lk and all its subsets")
print("Time taken: " + str(end-start))        

Enter minimum support: 
0.6

C1:
  {'MinorModality': 583, 'Medium Acousticness': 255, 'Cheerful': 1145, 'E': 158, 'MajorModality': 1389, 'High Instrumentalness': 21, 'C#': 214, 'Soft': 1702, 'A': 166, 'Medium Danceability': 1187, 'Live Audience': 9, 'C': 198, 'G#': 145, 'F#': 153, 'High Danceability': 725, 'Less Vocal': 413, 'High Energy': 1730, 'D#': 65, 'Low Instrumentalness': 1951, 'Sad': 827, 'A#': 138, 'Loud': 270, 'Low Acousticness': 1665, 'Low Tempo': 556, 'More Vocal': 1178, 'D': 211, 'B': 152, 'F': 151, 'High Acousticness': 52, 'Music & Speech': 381, 'Medium Tempo': 799, 'Low Energy': 242, 'Low Danceability': 60, 'No Audience': 1963, 'High Tempo': 617, 'G': 221}

L1:
  ['MajorModality', 'Soft', 'Medium Danceability', 'High Energy', 'Low Instrumentalness', 'Low Acousticness', 'No Audience']

C2 :
 [('MajorModality', 'Medium Danceability'), ('MajorModality', 'Low Instrumentalness'), ('MajorModality', 'No Audience'), ('Soft', 'High Energy'), ('Soft', 'Low Acousticness'), ('Medium

In [346]:
data1 = open('songs_1020.csv', 'r')
predata = preprocessing(data1)
n = len(predata)


print("Enter minimum support: ")
min_sup = float(input())

start = time.time()
freqsets = FreqIS(2,predata,min_sup)
end = time.time()

print("Therefore frequent patterns are itemsets present in last non-empty Lk and all its subsets")
print("Time taken: " + str(end-start))        

Enter minimum support: 
0.6

C1:
  {'MinorModality': 572, 'Medium Acousticness': 275, 'Cheerful': 976, 'E': 136, 'MajorModality': 1373, 'High Instrumentalness': 19, 'F#': 163, 'G#': 145, 'Low Instrumentalness': 1926, 'C#': 248, 'A': 180, 'Medium Danceability': 1128, 'Less Vocal': 281, 'C': 221, 'A#': 122, 'High Danceability': 784, 'Live Audience': 16, 'High Energy': 1628, 'D#': 55, 'Soft': 1591, 'Sad': 969, 'Music & Speech': 324, 'Low Acousticness': 1579, 'Low Tempo': 508, 'Medium Tempo': 782, 'D': 176, 'B': 158, 'F': 136, 'High Acousticness': 91, 'More Vocal': 1340, 'Low Energy': 317, 'Low Danceability': 33, 'No Audience': 1929, 'High Tempo': 655, 'Loud': 354, 'G': 205}

L1:
  ['MajorModality', 'Low Instrumentalness', 'High Energy', 'Soft', 'Low Acousticness', 'More Vocal', 'No Audience']

C2 :
 [('MajorModality', 'High Energy'), ('MajorModality', 'Low Acousticness'), ('MajorModality', 'No Audience'), ('Low Instrumentalness', 'Soft'), ('Low Instrumentalness', 'More Vocal'), ('High Ene

In [347]:
def preprocessing(data):
    lis = []
    for row in data: 
        row1 = ','.join([w.strip() for w in row.split(',')])
        row1 = row1.split(",")
        lis.append(row1)
    return lis, len(lis)

class FPTree:
    def __init__(self, name, freq, parentNode):
        self.name = name
        self.count = freq
        self.parent = parentNode
        self.nodeLink = None
        self.children = {}
    
    def increment(self, freq):
        self.count += freq
        
    def display(self, index = 1):
        print ('Level',index,'  '*index, self.name, ' ', self.count)
        for child in self.children.values():
            child.display(index+1)         

In [352]:
def createFPtree(data, min_support):
    freqcount = {}
    for row in data:
        for item in row:
            if item not in freqcount:
                freqcount[item] = 1
            else:
                freqcount[item] += 1

                
    for samp in list(freqcount):
        if freqcount[samp] < min_support:
            del(freqcount[samp])
    
    freqItemSet = list(freqcount.keys())
    print("Frequent Item Sets are:\n ",freqItemSet)
    print("\n")
    
    if len(freqItemSet) == 0:
        return None, None
    
    rDict = {}
    for obs in data:
        rDict[tuple(obs)] = 1        
    
    for obs in freqcount:
        freqcount[obs] = [freqcount[obs], None]
    
    rTree = FPTree('Null', 1, None)
    for trans, count in rDict.items():
        localDict= {}
        for item in trans:
            if item in freqItemSet:
                localDict[item] = freqcount[item][0]
        if len(localDict)>0:
            orderedItems = [k[0] for k in sorted(localDict.items(),key=lambda lambd: lambd[1], reverse=True)]
            update_Tree(orderedItems, rTree, freqcount, count)
    return rTree, freqcount
    

In [353]:
def update_Tree(items, inputTree, freqcount, count):
    if items[0] in inputTree.children:
        inputTree.children[items[0]].increment(count)
    else:
        inputTree.children[items[0]] = FPTree(items[0], count, inputTree)
        if freqcount[items[0]][1] == None: 
            freqcount[items[0]][1] = inputTree.children[items[0]]
        else:
            while (freqcount[items[0]][1].nodeLink != None):    
                freqcount[items[0]][1] = freqcount[items[0]][1].nodeLink
            freqcount[items[0]][1].nodeLink = inputTree.children[items[0]]

    if len(items) > 1:
        update_Tree(items[1::], inputTree.children[items[0]], freqcount, count)

        
def gotoRoot(leaf, pPath): 
    if leaf.parent != None:
        pPath.append(leaf.name)
        gotoRoot(leaf.parent, pPath)
        
def findPPath(basePatn, Node): 
    conPaths = {}
    while Node != None:
        pPath = []
        gotoRoot(Node, pPath)
        if len(pPath) > 1: 
            conPaths[tuple(pPath[1:])] = Node.count
        Node = Node.nodeLink
    return conPaths

def FPat(HeadTab):
    for key,value in HeadTab.items():
        p = []
        q = []
        count = 0
        X = findPPath(key, HeadTab[key][1])
        for j in X.keys():
            p.append(list(j))
        for j in X.values():
            q.append(j)       
        if len(p) > 0:
            result = set(p[0])
            for s in p[1:]:
                result.intersection_update(s)
            for i in q:
                count = count + i
            result = list(result)
            for i in range(len(result)):     
                print("\nPattern: <",key,',',result[i], ">:", count)
                print('\n')
            if len(result) >1 :    
                print("\nPattern: ", key, result,":", count)

In [354]:
data = open('songs_6070.csv', 'r')
predata,n = preprocessing(data)

print("Enter minimum support: ")
min_sup = float(input())
min_sup = int(min_sup * n)

print("Minimum support count is: ", min_sup)
start = time.time()
FPtree, HeadTab = createFPtree(predata, min_sup)
FPtree.display()
FPat(HeadTab)
end = time.time()
print('Time Taken: ',end-start)

Enter minimum support: 
0.6
Minimum support count is:  2336
Frequent Item Sets are:
  ['Cheerful', 'Medium Danceability', 'Low Instrumentalness', 'Loud', 'MajorModality', 'No Audience']


Level 1    Null   1
Level 2      MajorModality   2
Level 3        Cheerful   2
Level 4          Loud   2
Level 5            Medium Danceability   1
Level 2      Low Instrumentalness   57
Level 3        MajorModality   47
Level 4          Cheerful   36
Level 5            Medium Danceability   10
Level 5            Loud   23
Level 6              Medium Danceability   21
Level 4          Loud   11
Level 5            Medium Danceability   9
Level 3        Cheerful   9
Level 4          Medium Danceability   3
Level 4          Loud   6
Level 5            Medium Danceability   6
Level 3        Loud   1
Level 4          Medium Danceability   1
Level 2      Cheerful   1
Level 3        Loud   1
Level 4          Medium Danceability   1
Level 2      No Audience   2182
Level 3        Medium Danceability   1
Level 

In [355]:
data = open('songs_7080.csv', 'r')
predata,n = preprocessing(data)

print("Enter minimum support: ")
min_sup = float(input())
min_sup = int(min_sup * n)

print("Minimum support count is: ", min_sup)
start = time.time()
FPtree, HeadTab = createFPtree(predata, min_sup)
FPtree.display()
FPat(HeadTab)
end = time.time()
print('Time Taken: ',end-start)

Enter minimum support: 
0.6
Minimum support count is:  1858
Frequent Item Sets are:
  ['Cheerful', 'No Audience', 'Medium Danceability', 'High Energy', 'Low Instrumentalness', 'Loud', 'MajorModality']


Level 1    Null   1
Level 2      Low Instrumentalness   52
Level 3        MajorModality   9
Level 4          High Energy   9
Level 5            Medium Danceability   5
Level 3        High Energy   1
Level 4          Medium Danceability   1
Level 3        Cheerful   37
Level 4          MajorModality   10
Level 5            High Energy   10
Level 6              Medium Danceability   8
Level 4          High Energy   3
Level 5            Medium Danceability   3
Level 4          Loud   24
Level 5            Medium Danceability   1
Level 5            MajorModality   19
Level 6              Medium Danceability   7
Level 6              High Energy   11
Level 7                Medium Danceability   8
Level 5            High Energy   4
Level 6              Medium Danceability   3
Level 3        Lo

In [357]:
data = open('songs_8090.csv', 'r')
predata,n = preprocessing(data)

print("Enter minimum support: ")
min_sup = float(input())
min_sup = int(min_sup * n)

print("Minimum support count is: ", min_sup)
start = time.time()
FPtree, HeadTab = createFPtree(predata, min_sup)
FPtree.display()
FPat(HeadTab)
end = time.time()
print('Time Taken: ',end-start)

Enter minimum support: 
0.6
Minimum support count is:  1570
Frequent Item Sets are:
  ['Cheerful', 'No Audience', 'High Energy', 'Low Instrumentalness', 'Medium Tempo', 'Low Acousticness', 'Loud', 'MajorModality']


Level 1    Null   1
Level 2      Low Instrumentalness   41
Level 3        MajorModality   4
Level 4          Loud   4
Level 5            Medium Tempo   3
Level 3        High Energy   11
Level 4          Low Acousticness   7
Level 5            MajorModality   5
Level 6              Medium Tempo   2
Level 6              Loud   1
Level 5            Cheerful   2
Level 6              MajorModality   2
Level 4          Cheerful   2
Level 5            MajorModality   2
Level 6              Loud   1
Level 7                Medium Tempo   1
Level 4          MajorModality   2
Level 5            Loud   2
Level 3        Low Acousticness   26
Level 4          MajorModality   2
Level 5            Loud   2
Level 6              Medium Tempo   2
Level 4          High Energy   24
Level 5     

In [358]:
data = open('songs_9000.csv', 'r')
predata,n = preprocessing(data)

print("Enter minimum support: ")
min_sup = float(input())
min_sup = int(min_sup * n)

print("Minimum support count is: ", min_sup)
start = time.time()
FPtree, HeadTab = createFPtree(predata, min_sup)
FPtree.display()
FPat(HeadTab)
end = time.time()
print('Time Taken: ',end-start)

Enter minimum support: 
0.6
Minimum support count is:  1195
Frequent Item Sets are:
  ['Cheerful', 'No Audience', 'Low Instrumentalness', 'High Energy', 'Low Acousticness', 'MajorModality']


Level 1    Null   1
Level 2      Low Instrumentalness   19
Level 3        Low Acousticness   17
Level 4          High Energy   17
Level 5            MajorModality   7
Level 6              Cheerful   6
Level 5            Cheerful   9
Level 3        High Energy   1
Level 4          Cheerful   1
Level 3        MajorModality   1
Level 2      No Audience   1187
Level 3        Low Acousticness   41
Level 4          MajorModality   1
Level 4          High Energy   40
Level 5            MajorModality   28
Level 6              Cheerful   20
Level 5            Cheerful   8
Level 3        Low Instrumentalness   1144
Level 4          Low Acousticness   830
Level 5            MajorModality   109
Level 6              Cheerful   38
Level 5            High Energy   639
Level 6              MajorModality   397
Lev

In [359]:
data = open('songs_0010.csv', 'r')
predata,n = preprocessing(data)

print("Enter minimum support: ")
min_sup = float(input())
min_sup = int(min_sup * n)

print("Minimum support count is: ", min_sup)
start = time.time()
FPtree, HeadTab = createFPtree(predata, min_sup)
FPtree.display()
FPat(HeadTab)
end = time.time()
print('Time Taken: ',end-start)

Enter minimum support: 
0.6
Minimum support count is:  1183
Frequent Item Sets are:
  ['MajorModality', 'Soft', 'Medium Danceability', 'High Energy', 'Low Instrumentalness', 'Low Acousticness', 'No Audience']


Level 1    Null   1
Level 2      Low Instrumentalness   9
Level 3        High Energy   8
Level 4          Low Acousticness   1
Level 5            MajorModality   1
Level 6              Medium Danceability   1
Level 4          Soft   7
Level 5            Low Acousticness   7
Level 6              MajorModality   5
Level 7                Medium Danceability   4
Level 3        Soft   1
Level 4          Low Acousticness   1
Level 2      No Audience   967
Level 3        Low Acousticness   4
Level 4          MajorModality   3
Level 5            Medium Danceability   2
Level 3        Low Instrumentalness   946
Level 4          Medium Danceability   6
Level 4          MajorModality   60
Level 5            Medium Danceability   46
Level 4          High Energy   738
Level 5            Medi

In [360]:
data = open('songs_1020.csv', 'r')
predata,n = preprocessing(data)

print("Enter minimum support: ")
min_sup = float(input())
min_sup = int(min_sup * n)

print("Minimum support count is: ", min_sup)
start = time.time()
FPtree, HeadTab = createFPtree(predata, min_sup)
FPtree.display()
FPat(HeadTab)
end = time.time()
print('Time Taken: ',end-start)

Enter minimum support: 
0.6
Minimum support count is:  1167
Frequent Item Sets are:
  ['MajorModality', 'Low Instrumentalness', 'High Energy', 'Soft', 'Low Acousticness', 'More Vocal', 'No Audience']


Level 1    Null   1
Level 2      Low Instrumentalness   15
Level 3        High Energy   15
Level 4          Low Acousticness   2
Level 5            MajorModality   1
Level 6              More Vocal   1
Level 4          Soft   13
Level 5            Low Acousticness   10
Level 6              MajorModality   7
Level 7                More Vocal   6
Level 6              More Vocal   3
Level 5            MajorModality   3
Level 6              More Vocal   3
Level 2      No Audience   1008
Level 3        Soft   1
Level 4          Low Acousticness   1
Level 5            More Vocal   1
Level 3        MajorModality   6
Level 4          More Vocal   5
Level 3        Low Instrumentalness   990
Level 4          Low Acousticness   80
Level 5            MajorModality   48
Level 6              More Voca

In [361]:
data = pd.read_csv('Complete_project_data3.csv')
data.head()

,Artist,Track,Year,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Adam Ant,Friend Or Foe,1982,0.535,0.970,9.0,-3.396,1.0,0.0986,0.22600,0.000009,0.5780,0.669,91.890
1,Jeff And Sheri Easter,Ordinary Day,2006,0.528,0.778,10.0,-5.869,1.0,0.0301,0.10800,0.000001,0.0820,0.527,153.861
2,Rated R,Da Ghetto Psychic,2003,0.914,0.731,0.0,-5.572,1.0,0.1620,0.00927,0.000009,0.0836,0.593,105.015
3,Tweeterfriendly Music,Gin & Phonic,2003,0.534,0.733,7.0,-8.555,1.0,0.0409,0.06210,0.004500,0.4160,0.677,100.079
4,Planet P Project,Pink World,1984,0.292,0.847,9.0,-5.483,1.0,0.1250,0.00199,0.002390,0.0802,0.514,85.656


In [363]:
data['Hit'] = 0

In [364]:
data.head()

,Artist,Track,Year,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Hit
0,Adam Ant,Friend Or Foe,1982,0.535,0.970,9.0,-3.396,1.0,0.0986,0.22600,0.000009,0.5780,0.669,91.890,0
1,Jeff And Sheri Easter,Ordinary Day,2006,0.528,0.778,10.0,-5.869,1.0,0.0301,0.10800,0.000001,0.0820,0.527,153.861,0
2,Rated R,Da Ghetto Psychic,2003,0.914,0.731,0.0,-5.572,1.0,0.1620,0.00927,0.000009,0.0836,0.593,105.015,0
3,Tweeterfriendly Music,Gin & Phonic,2003,0.534,0.733,7.0,-8.555,1.0,0.0409,0.06210,0.004500,0.4160,0.677,100.079,0
4,Planet P Project,Pink World,1984,0.292,0.847,9.0,-5.483,1.0,0.1250,0.00199,0.002390,0.0802,0.514,85.656,0


In [365]:
data.to_csv('Complete_project_data.csv')

In [366]:
data1 = pd.read_csv('FinalData_1.csv')
data1.head()

,Artist,Track,Year,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Marty Robbins,El Paso,1960,0.654,0.452,2,-9.709,1,0.0300,0.835,0.000029,0.160,0.691,106.662
1,Frankie Avalon,Why,1960,0.510,0.349,5,-8.677,1,0.0264,0.762,0.000000,0.130,0.586,94.267
2,Miss Toni Fisher,The Big Hurt,1960,0.624,0.557,0,-8.309,1,0.0299,0.783,0.000000,0.173,0.676,123.919
3,Johnny Preston,Running Bear,1960,0.772,0.297,5,-14.679,0,0.0530,0.854,0.000008,0.125,0.822,119.987
4,Guy Mitchell,Heartaches By The Number,1960,0.584,0.589,5,-11.203,1,0.0718,0.463,0.000015,0.069,0.842,172.484


In [367]:
data1['Hit'] = 1

In [368]:
data1.head()

,Artist,Track,Year,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Hit
0,Marty Robbins,El Paso,1960,0.654,0.452,2,-9.709,1,0.0300,0.835,0.000029,0.160,0.691,106.662,1
1,Frankie Avalon,Why,1960,0.510,0.349,5,-8.677,1,0.0264,0.762,0.000000,0.130,0.586,94.267,1
2,Miss Toni Fisher,The Big Hurt,1960,0.624,0.557,0,-8.309,1,0.0299,0.783,0.000000,0.173,0.676,123.919,1
3,Johnny Preston,Running Bear,1960,0.772,0.297,5,-14.679,0,0.0530,0.854,0.000008,0.125,0.822,119.987,1
4,Guy Mitchell,Heartaches By The Number,1960,0.584,0.589,5,-11.203,1,0.0718,0.463,0.000015,0.069,0.842,172.484,1


In [369]:
data1.to_csv('Complete.csv')

In [383]:
finaldata = pd.read_csv('Complete_project_data.csv')
finaldata= finaldata.drop(['Unnamed: 0'], axis = 1)
finaldata.head()

,Artist,Track,Year,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Hit
0,Adam Ant,Friend Or Foe,1982,0.535,0.970,9.0,-3.396,1.0,0.0986,0.22600,0.000009,0.5780,0.669,91.890,0.0
1,Jeff And Sheri Easter,Ordinary Day,2006,0.528,0.778,10.0,-5.869,1.0,0.0301,0.10800,0.000001,0.0820,0.527,153.861,0.0
2,Rated R,Da Ghetto Psychic,2003,0.914,0.731,0.0,-5.572,1.0,0.1620,0.00927,0.000009,0.0836,0.593,105.015,0.0
3,Tweeterfriendly Music,Gin & Phonic,2003,0.534,0.733,7.0,-8.555,1.0,0.0409,0.06210,0.004500,0.4160,0.677,100.079,0.0
4,Planet P Project,Pink World,1984,0.292,0.847,9.0,-5.483,1.0,0.1250,0.00199,0.002390,0.0802,0.514,85.656,0.0


In [381]:
finaldata.drop_duplicates(subset ="Track", 
                     keep = False, inplace = True)

In [384]:
finaldata= finaldata.drop(['Artist','Year', 'Track'], axis = 1)

In [415]:
finaldata.to_csv('ModelData.csv', index= False)

In [432]:
df = pd.read_csv('ModelData.csv')
df.reset_index()
X = df.iloc[:, :11]
Y = df.iloc[:, 11]
Y = Y.astype('int64')

In [495]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

In [496]:
clf = SVC(kernel='rbf', C=0.5)
Y_pred = cross_val_predict(clf, X, Y, cv=10)
target_names = ['Hit', 'Not Hit']
print(classification_report(Y, Y_pred, target_names=target_names))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 5, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

        Hit       0.77      0.61      0.68      3764
    Not Hit       0.68      0.82      0.74      3767

avg / total       0.72      0.71      0.71      7535



In [483]:
from sklearn.linear_model import LogisticRegression

In [497]:
logisticRegr = LogisticRegression(penalty='l2')
Y_pred = cross_val_predict(logisticRegr, X, Y, cv=10)
target_names = ['Hit', 'Not Hit']
print(classification_report(Y, Y_pred, target_names=target_names))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

        Hit       0.82      0.70      0.76      3764
    Not Hit       0.74      0.85      0.79      3767

avg / total       0.78      0.77      0.77      7535



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 5, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [498]:
from sklearn.ensemble import RandomForestClassifier

In [501]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42, max_depth = 6, min_samples_split = 70)
Y_pred = cross_val_predict(rf, X, Y, cv=10)
target_names = ['Hit', 'Not Hit']
print(classification_report(Y, Y_pred, target_names=target_names))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

        Hit       0.83      0.71      0.77      3764
    Not Hit       0.75      0.85      0.80      3767

avg / total       0.79      0.78      0.78      7535



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 5, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [502]:
from xgboost import XGBClassifier

In [504]:
model = XGBClassifier(subsample = 0.7, reg_lambda = 0.8)
Y_pred = cross_val_predict(model, X, Y, cv=10)
target_names = ['Hit', 'Not Hit']
print(classification_report(Y, Y_pred, target_names=target_names))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this 

             precision    recall  f1-score   support

        Hit       0.83      0.75      0.79      3764
    Not Hit       0.77      0.85      0.81      3767

avg / total       0.80      0.80      0.80      7535



/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 5, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [506]:
from sklearn.neural_network import MLPClassifier

In [507]:
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10, 10), max_iter=1000, activation = 'relu', learning_rate = 'adaptive' )
Y_pred = cross_val_predict(mlp, X, Y, cv=10)
target_names = ['Hit', 'Not Hit']
print(classification_report(Y, Y_pred, target_names=target_names))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

        Hit       0.76      0.70      0.73      3764
    Not Hit       0.72      0.78      0.75      3767

avg / total       0.74      0.74      0.74      7535



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 5, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [508]:
scores = cross_val_score(clf, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.71 (+/- 0.07)


In [509]:
scores = cross_val_score(logisticRegr, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.77 (+/- 0.06)


In [510]:
scores = cross_val_score(rf, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.78 (+/- 0.04)


In [512]:
scores = cross_val_score(model, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this 

Accuracy: 0.80 (+/- 0.05)


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [511]:
scores = cross_val_score(mlp, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.75 (+/- 0.07)
